In [1]:
import math
import itertools
import numpy as np

import hoomd
import gsd.hoomd
import coxeter

#### particle vertices, faces, and insphere radius

In [2]:
lattice = gsd.hoomd.open('./DATA/lattice.gsd')
verts = lattice[0].particles.type_shapes[0]['vertices']
particle = coxeter.shapes.ConvexPolyhedron(verts)
faces = particle.faces
sigma = 2*particle.insphere_from_center.radius

sphere = coxeter.shapes.Ellipsoid(0.5, 0.5, 0.5)
sigma2 = sphere.maximal_bounded_sphere_radius * 2

#### Parameters

In [3]:
# integration timestep
dt=0.0005

# coupling
kT_init=1.0
kT_second=2.0
tau=100*dt

#### simultaion

In [4]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)

sim.create_state_from_gsd(filename='./DATA/lattice.gsd')
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), 
                                      kT=kT_init)

#### potential

In [5]:
# neighboring list
nl = hoomd.md.nlist.Cell(buffer=0.4)

# potential
r_cut_A = 2*particle.circumsphere_from_center.radius + 0.15*sigma + 0.167*sigma*2
r_cut_B = sigma2 + 0.15*sigma2
r_cut_AB = r_cut_A + r_cut_B + 2**(1/6)*0.15*(sigma+sigma2)

alj = hoomd.md.pair.aniso.ALJ(nl)
alj.r_cut[('A', 'A')] = r_cut_A
alj.params[('A', 'A')] = dict(epsilon=0.1, 
                              sigma_i=sigma, 
                              sigma_j=sigma, 
                              alpha=0)

alj.r_cut[('B', 'B')] = r_cut_B
alj.params[('B', 'B')] = dict(epsilon=0.1, 
                              sigma_i=sigma2, 
                              sigma_j=sigma2, 
                              alpha=0)

alj.r_cut[('A', 'B')] = r_cut_AB
alj.params[('A', 'B')] = dict(epsilon=0.1, 
                              sigma_i=sigma, 
                              sigma_j=sigma2, 
                              alpha=2)

alj.shape['A'] = dict(vertices=verts, 
                      faces=faces, 
                      rounding_radii=0.167*sigma) # rounding_radii=0.15*(sigma/2)
alj.shape['B'] = dict(vertices = [], 
                      faces = [], 
                      rounding_radii = (sphere.a, sphere.b, sphere.c))


#### thermodynamic ensemble

In [6]:
t_duration=int(2e4)

kT = hoomd.variant.Cycle(A=kT_init, 
                         B=kT_second, 
                         t_start=sim.timestep, 
                         t_A=t_duration, 
                         t_AB=t_duration, 
                         t_B=t_duration, 
                         t_BA=t_duration)

nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), 
                           kT=kT, 
                           tau=tau)

#### integrator

In [7]:
integrator = hoomd.md.Integrator(dt=dt, 
                                 methods=[nvt],
                                 forces=[alj],
                                 integrate_rotational_dof=True)

#### writer

In [8]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())

logger = hoomd.logging.Logger()
logger.add(sim, quantities=['timestep', 'walltime'])
logger.add(alj, quantities=['type_shapes'])
logger.add(thermodynamic_properties)

In [9]:
gsd_writer = hoomd.write.GSD(filename='./DATA/anneal.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             log=logger,
                             mode='wb')

In [10]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = integrator
sim.operations.computes.append(thermodynamic_properties)

#### run

In [11]:
sim.run(0)
alj.type_shapes

[{'type': 'ConvexPolyhedron',
  'rounding_radius': 0.484,
  'vertices': [[1, 1, 1],
   [1, 1, -1],
   [1, -1, 1],
   [-1, 1, 1],
   [-1, 1, -1],
   [1, -1, -1],
   [-1, -1, 1],
   [-1, -1, -1]]},
 {'type': 'Ellipsoid', 'a': 0.575, 'b': 0.575, 'c': 0.575}]

In [12]:
sim.run(t_duration*5)